<a href="https://colab.research.google.com/github/patrickongwong/ColabProjects/blob/master/Copy_of_Watchlist_%2B_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code

In [ ]:
import plotly
import requests
import numpy as np
import pandas as pd
!pip install yfinance
import yfinance as yf

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=4ff7c7b0e712285f77c70bdcdc00a9821ab344d1a54b2f850084ffc8acf38883
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [ ]:
# yFinance Important functions
def price_taker(tickers, per):
    data = pd.DataFrame()
    for ticker in tickers:

        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period=per)
            hist['Ticker'] = ticker
            data = data.append(hist.resample('D').fillna("pad"))
        except:
            print("error on ", ticker)
            pass
    return data

# Formula for total returns for a time period
def total_returns(start, end, df, ytd):
    df = df.loc[(df.index > start) & (df.index <= end)]
    total_divs = df['Dividends'].sum()
    if ytd == True:
        start_price = df['Open'].iloc[0]
    else:
        start_price = df['Close'].iloc[0]
    
    end_price = df['Close'].iloc[-1]
    answer = total_divs/start_price + end_price/start_price -1 
    return answer

def price_returns(start, end, df, ytd):
    df = df.loc[(df.index > start) & (df.index <= end)]

    if ytd == True:
        start_price = df['Open'].iloc[0]
    else:
        start_price = df['Close'].iloc[0]
    
    end_price = df['Close'].iloc[-1]
    answer = end_price/start_price -1
    return answer

def div_returns(start, end, df, ytd):
    df = df.loc[(df.index > start) & (df.index <= end)]
    total_divs = df['Dividends'].sum()

    if ytd == True:
        start_price = df['Open'].iloc[0]
    else:
        start_price = df['Close'].iloc[0]
    
    answer = total_divs/start_price
    return answer


In [ ]:
!pip install gspread_formatting
import gspread
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.create('New Sheet').sheet1


In [ ]:
today = '2020-08-18'

start = pd.Timestamp('2020-01-01')
end = pd.Timestamp(today)
end_less_wk = end - np.timedelta64(7,'D')
end_less_day = end - np.timedelta64(2,'D')

global_dict = {'MSCI World' : 'MWL=F',
               'MSCI EM'    : 'MME=F'}

US_dict = {'S&P 500'    : '^GSPC',
           'Dow Jones Index' : '^DJI',
           'Russell 2000' : '^RUT',
           'NASDAQ Comp' : '^IXIC'}

EU_dict = {'Stoxx 50'    : '^STOXX50E', #Stoxx50
           'EURONEXT 100' : '^N100', #EURONEXT 100
           'Germany' : '^GDAXI', #DAX
           'Switzerland' : '^SSMI', #Swiss Index
           'France' : '^FCHI', #CAC 40
           'United Kingdoms' : '^FTSE', #FTSE 100
           #'Italy' : 'FTSEMIBN.MI', #FTSE MIB Net Total Returns
           'Spain'  : '^IBEX' #IBEX
           }

Asia_dict = {'Japan' : '^N225', #Nikkei 225
             #'China CSI 300': '000300.SS', #China CSI 300
             'China H-Shares': '^HSCE', #China H-Shares
             'Hong Kong': '^HSI', #Hang Seng Index
             'Shenzhen Comp': '399001.SZ', #Shenzhen Composite
             'Australia': '^AXJO', #ASX 200
             'Philippines': 'PSEI.PS', #PSEI
             'Singapore':'^STI', #STI Index
             'Indonesia':'^JKSE', #Jakarta Composite Index
             'India Sensex':'^BSESN', #S&P BSE SENSEX
             'Malaysia':'^KLSE', #FTSE Bursa Malaysia KLCI
             #'Vietnam':'FVTT.FGI', #FTSE Vietnam
             #'Thailand':'^SET.BK', #SET Index
             'Taiwan':'^TWII', #TSEC Weighted Index
             'Korea':'^KS11' #KOSPI Composite Index
            }

comm_dict = {'Gold': 'GC=F', # Gold Futures
             'Silver': 'SI=F', # Silver Futures
             'Oil (WTI)':'CL=F', # Crude Oil Futures, WTI
             'Oil (Brent)':'BZ=F', # Oil Futures, Brent
             'Platinum':'PL=F' #Platinum Futures
             }

vol_dict = {'S&P 500 VIX Index':'^VIX'
            }

us_ind_dict = {'Technology':'XLK',
               'Energy':'XLE',
               'Financials':'XLF',
               'Healthcare':'XLV',
               'Cons Disc':'XLY',
               'Industrial':'XLI',
               'Material':'XLB',
               'Utilities':'XLU',
               'Cons Staples':'XLP',
               'Real Estate':'IYR'
              }

us_fact_dict = {'Largecap Growth':'VUG',
                'Largecap Value':'VTV',
                'Largecap Blend':'VV',
                'Midcap Growth':'VOT',
                'Midcap Value':'VOE',
                'Midcap Blend':'VO',
                'Smallcap Growth':'VBK',
                'Smallcap Value':'VBR',
                'Smallcap Blend':'VB'
              }

fmt5 = cellFormat(
    numberFormat = numberFormat(type='PERCENT', pattern='0.00%')
    )

def get_key(val, dict1): 
    for key, value in dict1.items(): 
         if val == value: 
             return key 
  
    return "key doesn't exist"

def dict_returns_gsheet(dictionary, rowz, columnz):
  global start
  global end
  global end_less_wk
  global end_less_day

  ans = pd.DataFrame()

  df = price_taker(dictionary.values(), 'ytd')
  df['Ticker'] = df['Ticker'].apply(lambda x : get_key(x , dictionary))
  ytd_returns = df.groupby('Ticker').apply(lambda x: total_returns(start, end, x, True))
  wk_returns = df.groupby('Ticker').apply(lambda x: total_returns(end_less_wk, end, x, False))
  day_returns = df.groupby('Ticker').apply(lambda x: total_returns(end_less_day, end, x, False))
  end_price = df.groupby('Ticker').apply(lambda x: x['Close'].iloc[-1])

  ans['last'] = end_price
  ans['1D%'] = day_returns
  ans['1Wk%'] = wk_returns
  ans['YTD%'] = ytd_returns

  ans = ans.sort_values(by=['1Wk%'], ascending = False)

  set_with_dataframe(worksheet, ans, include_index=True, row=rowz, col=columnz, include_column_header = False)

  return ans

worksheet.update_cell(2, 3, "World Markets: "+str(today))
dict_returns_gsheet(global_dict, 5, 3)
dict_returns_gsheet(US_dict, 9, 3)
dict_returns_gsheet(us_ind_dict, 15, 3)
dict_returns_gsheet(us_fact_dict, 27, 3)


dict_returns_gsheet(EU_dict, 5, 9)
dict_returns_gsheet(Asia_dict, 14, 9)
dict_returns_gsheet(comm_dict, 28, 9)

dict_returns_gsheet(vol_dict, 35, 9)

format_cell_range(worksheet, 'E:G', fmt5)
format_cell_range(worksheet, 'K:M', fmt5)


{'replies': [{}],
 'spreadsheetId': '1XNAKOJVvp-QARbPN9HN4_jRRF54uESG3c_pkLuV0ptA'}